In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


# Keras for Neural Networks - Guided Example

Here we're going to work through a classic Machine Learning problem - digit recognition. This data is referred to as the MNIST dataset, which stands for Modified National Institute of Standards and Technology, and represents probably the most used dataset in the world for advanced machine learning techniques (though the iris dataset would be a close second). Here we're forgoing a more business focussed dataset for a few reasons. Firstly, this dataset is the most written about dataset in these topics - you'll easily find other guides using pure TensorFlow or other tools like Theano to solve the same problem with the same class of models. Similarly, you can also easily find several different kinds of neural networks being used to solve this problem. This will be valuable as you try to expand your knowledge of different kinds of layers and combinations.

We'll be building our code off of the examples provided in the Keras documentation, and found in full on its [creator's github](https://github.com/fchollet/keras/tree/master/examples). 

Our goal here will be simple but multifaceted. Overall we are going to use the MNIST dataset and neural networks to classify handwritten numbers as the proper digits. This will be thought of as a multi-class classification problem, specifically with 10 classes (one for each possible digit).

However, we will use this to teach a few new kinds of neural network compositions, creating three different styles of network and discussing their relative advantages and disadvantages. Through this we will delve a little deeper into neural network theory.

But before we go too far, let's actually look at the data.

## MNIST DATA

In [2]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ..., 
       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, 

When you look at this data you'll notice its organization structure is not images. We don't actually see any pictures of digits here. Instead, what we have is values of pixels, a simple way of converting images into numeric data on which we can train a model.

However, this still doesn't look like most of the data we've worked with previously. It's not a single table, but rather a different, higher dimensionality structure. It is often described as a set of clouds, each cloud representing an image. The cloud contains columns of values, representing the darkness of pixels. That's great, but not an easy or meaningful dataset on which to directly train a model. The darkness of the second pixel in the third column isn't likely linearly related to likelihood the cloud represents a certain digit. Instead, we need to find meaningful patterns within our clouds, creating models off of those patterns.

This is exactly what neural networks are good at. Multiple layers will allow us to transform this clouds full of values into meaningful vectors containing the information we need to be able to create a model, admittedly in an unlabeled and unsupervised fashion. Our output, however, will be labels for each of the clouds, giving us predictions as to what digit they are meant to represent.

Let's get started.

## Multi Layer Perceptron

Let's start with a kind of neural network we've seen before: a multi-layer perceptron. Recall from our previous neural networks sections that this is a set of perceptron models organized into layers, one layer feeding into the next.

To do this, we will first need to reshape our data into flat vectors for each digit. We'll also need to convert our outcome to a matrix of binary variables, rather than the digit.

In [5]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


Great. Now we can create our model. We'll do this using dense layers and dropouts. Dense layers are simply fully connected layers with a given number of perceptrons. Dropout drops a certain portion of our perceptrons in order to prevent overfitting. Our activation function, `relu` stands for Rectified Linear Unit, which is standard but can be read about more [here](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).

In [6]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


Now we have a model. This we can use to accomplish our wildest dreams of data modeling, or at least predict some digits from pixel data. To do that we will use epochs, effectively iterations of the model, improving based on what it learned previously. Batch size is the number of samples to use in each step improving the model and will affect speed, but also slightly negatively impact accuracy (learning in bigger steps will affect what your model learns).

Note that we are going with 64 perceptron wide layers, this is relatively arbitrary, though units within the $2^x$ series will parallelize more efficiently. Also note that our number of parameters is the product of our input width plus one and our layer width. This reflects the number of weights we're creating in that layer.

In [7]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s - loss: 0.4332 - acc: 0.8756 - val_loss: 0.1975 - val_acc: 0.9397
Epoch 2/10
60000/60000 [==============================] - 1s - loss: 0.2024 - acc: 0.9400 - val_loss: 0.1342 - val_acc: 0.9581
Epoch 3/10
60000/60000 [==============================] - 1s - loss: 0.1582 - acc: 0.9523 - val_loss: 0.1170 - val_acc: 0.9644
Epoch 4/10
60000/60000 [==============================] - 1s - loss: 0.1307 - acc: 0.9608 - val_loss: 0.1018 - val_acc: 0.9686
Epoch 5/10
60000/60000 [==============================] - 1s - loss: 0.1168 - acc: 0.9645 - val_loss: 0.0983 - val_acc: 0.9715
Epoch 6/10
60000/60000 [==============================] - 1s - loss: 0.1041 - acc: 0.9685 - val_loss: 0.0906 - val_acc: 0.9727
Epoch 7/10
60000/60000 [==============================] - 1s - loss: 0.0947 - acc: 0.9710 - val_loss: 0.0909 - val_acc: 0.9723
Epoch 8/10
60000/60000 [==============================] - 1s 

That did impressively well for such a simple neural network, with each epoch training in about 1 second on this machine and giving us an accuracy in the high 90's. But what else can we do? Let's let our model get much more complicated by introducting convolution.

## Convolutional Neural Networks

Before we go any further, do you recall that we've talked about how complex neural networks can get, and the degree of computational complexity that entails? Well, here we're going to finally truly experience that complexity, so be careful about rerunning this code. It will take some serious time (potentially on the order of hours) to run.

Now that that's out of the way, let's talk convolution. First, a simple definition. Convolution basically takes your data and creates overlapping subsegments testing for a given feature in a set of spaces and upon which it develops its model.

Let's extend that definition since it's incredibly dense.

First, you have to define a shape of your input data. This can theoretically be in any number of dimensions, though for our image example we will use 2d, since images are in two dimensions. This is also why you'll see 2D in some of our layer definitions (though more on that later). Our first chunk of code after loading the data does this reshaping (with a conditional on the data format).

Over that shaped data, we then create our tiles, also called __kernels__. These kernels are like little windows, that will look over subsets of the data of a given size. In the example below we create 3x3 kernels, which run overlapping over the whole 28x28 input looking for features. That is the convolutional layer, a way of searching for a subpattern over the whole of the image. We can chain multiple of these convolutional layers together, with the below example having two.

Next comes a pooling layer. This is a _downsampling_ technique, which effectively serves to reduce sample size and simplify later processes. For each value generated by our convolutional layers, it looks over the grid in _non_-overlapping segments and takes the maximum value of those outputs. It's not the feautres exact location then that matters, but its approximate or relative location. After pooling you will want to flatten the data back out, so that it can be put into dense layers as we did in MLP.

In [9]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 123s - loss: 0.3270 - acc: 0.9011 - val_loss: 0.0720 - val_acc: 0.9782
Epoch 2/10
60000/60000 [==============================] - 124s - loss: 0.1113 - acc: 0.9677 - val_loss: 0.0517 - val_acc: 0.9831
Epoch 3/10
60000/60000 [==============================] - 124s - loss: 0.0880 - acc: 0.9738 - val_loss: 0.0468 - val_acc: 0.9846
Epoch 4/10
60000/60000 [==============================] - 121s - loss: 0.0715 - acc: 0.9791 - val_loss: 0.0393 - val_acc: 0.9863
Epoch 5/10
60000/60000 [==============================] - 120s - loss: 0.0632 - acc: 0.9815 - val_loss: 0.0350 - val_acc: 0.9874
Epoch 6/10
60000/60000 [==============================] - 124s - loss: 0.0569 - acc: 0.9827 - val_loss: 0.0334 - val_acc: 0.9884
Epoch 7/10
60000/60000 [==============================] - 127s - loss: 0.0516 - acc: 0.9840 - val_loss:

Now that is incredibly impressive accuracy. 99% is really exceptional, but it did take a long time to get there. Such are the costs of convolution.

There is one more classic construction of a neural network: Recurrent, which we'll give quick mention.

## Hierarchical Recurrrent Neural Networks

So far when we've talked about neural networks we've talked about them as feedforward: data flows in one direction until it reaches the end. Recurrent neural networks do not obey that directional logic, instead letting the data cycle through the network.

However, to do this we have to abandon the sequential model building we've done so far and things can get much more complicated. You have to use recurrent layers and often time distribution (which handles the extra dimension created through the LTSM layer, as a time dimension) to get these things running. You can find an example of a hierarchical recurrent network below (via the link [here](https://github.com/fchollet/keras/blob/master/examples/mnist_hierarchical_rnn.py)). When you get comfortable with networks as they exist in Keras for both convolution and MLP, start exploring recurrence. Note that this will take an even longer time than the previous ones should you choose to run it again.

In [10]:

# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 352s - loss: 0.9851 - acc: 0.6618 - val_loss: 0.5004 - val_acc: 0.8216
Epoch 2/3
60000/60000 [==============================] - 350s - loss: 0.4009 - acc: 0.8709 - val_loss: 0.3192 - val_acc: 0.8997
Epoch 3/3
60000/60000 [==============================] - 346s - loss: 0.2343 - acc: 0.9284 - val_loss: 0.2199 - val_acc: 0.9293
Test loss: 0.219856005466
Test accuracy: 0.9293


You should now be comfortable building some neural networks, but let's see if you can improve them!

# Drill: 99% MLP

We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or concolutional neural nets?

In [23]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='linear', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 64)                50240     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s - loss: 0.4110 - acc: 0.8801 - val_loss: 0.2131 - val_acc: 0.9379
Epoch 2/10
60000/60000 [==============================] - 1s - loss: 0.2335 - acc: 0.9316 - val_loss: 0.1664 - val_acc: 0.9504
Epoch 3/10
60000/60000 [==============================] - 1s - loss: 0.1943 - acc: 0.9428 - val_loss: 0.1554 - val_acc: 0.9534
Epoch 4/10
60000/60000 [==============================] - 1s - loss: 0.1753 - acc: 0.9474 - val_loss: 0.1445 - val_acc: 0.9581
Epoch 5/10
60000/60000 [==============================] - 1s - loss: 0.1569 - acc: 0.9525 - val_loss: 0.1274 - val_acc: 0.9629
Epoch 6/10
60000/60000 [==============================] - 1s - loss: 0.1476 - acc: 0.9553 - val_loss: 0.1225 - val_acc: 0.9633
Epoch 7/10
60000/60000 [==============================] - 1s - loss: 0.1381 - acc: 0.9577 - val_loss: 0.1169 - val_acc: 0.9664
Epoch 8/10
60000/60000 [==============================] - 1s 

# Results of Exploratory Analysis:

activation = relu / elu / softmax - Test loss: 0.0870043254456 Test accuracy: 0.9753
activation = relu / selu / softmax - Test loss: 0.0847957983427 Test accuracy: 0.9771
activation = selu / selu / softmax - Test loss: 0.0937839540551 Test accuracy: 0.9726
activation = tanh / relu / softmax - Test loss: 0.0891504549881 Test accuracy: 0.9725
activation = sigmoid / relu / softmax - Test loss: 0.0988173509547 Test accuracy: 0.9696
activation = hard_sigmoid / relu / softmax - Test loss: 0.100846571798 Test accuracy: 0.9695
activation = linear / relu / softmax - Test loss: 0.111678240961 Test accuracy: 0.9696

In [37]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(120, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 120)               94200     
_________________________________________________________________
dropout_31 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 120)               14520     
_________________________________________________________________
dropout_32 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 10)                1210      
Total params: 109,930
Trainable params: 109,930
Non-trainable params: 0
_________________________________________________________________


In [43]:
history = model.fit(x_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s - loss: 0.0074 - acc: 0.9978 - val_loss: 0.1238 - val_acc: 0.9804
Epoch 2/10
60000/60000 [==============================] - 1s - loss: 0.0080 - acc: 0.9977 - val_loss: 0.1270 - val_acc: 0.9805
Epoch 3/10
60000/60000 [==============================] - 1s - loss: 0.0085 - acc: 0.9978 - val_loss: 0.1361 - val_acc: 0.9795
Epoch 4/10
60000/60000 [==============================] - 1s - loss: 0.0080 - acc: 0.9977 - val_loss: 0.1330 - val_acc: 0.9801
Epoch 5/10
60000/60000 [==============================] - 1s - loss: 0.0071 - acc: 0.9978 - val_loss: 0.1237 - val_acc: 0.9818
Epoch 6/10
60000/60000 [==============================] - 1s - loss: 0.0087 - acc: 0.9977 - val_loss: 0.1176 - val_acc: 0.9812
Epoch 7/10
60000/60000 [==============================] - 1s - loss: 0.0085 - acc: 0.9977 - val_loss: 0.1223 - val_acc: 0.9817
Epoch 8/10
60000/60000 [==============================] - 1s 

In [44]:
#activation = relu / selu / softmax 

model = Sequential()

model.add(Dense(120, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(120, activation='selu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 120)               94200     
_________________________________________________________________
dropout_33 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 120)               14520     
_________________________________________________________________
dropout_34 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 10)                1210      
Total params: 109,930
Trainable params: 109,930
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model.fit(x_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s - loss: 0.3700 - acc: 0.8907 - val_loss: 0.1676 - val_acc: 0.9478
Epoch 2/10
60000/60000 [==============================] - 2s - loss: 0.1584 - acc: 0.9528 - val_loss: 0.1207 - val_acc: 0.9628
Epoch 3/10
60000/60000 [==============================] - 1s - loss: 0.1136 - acc: 0.9648 - val_loss: 0.0993 - val_acc: 0.9699
Epoch 4/10
60000/60000 [==============================] - 1s - loss: 0.0896 - acc: 0.9725 - val_loss: 0.0877 - val_acc: 0.9734
Epoch 5/10
60000/60000 [==============================] - 2s - loss: 0.0753 - acc: 0.9766 - val_loss: 0.0865 - val_acc: 0.9743
Epoch 6/10
60000/60000 [==============================] - 1s - loss: 0.0640 - acc: 0.9800 - val_loss: 0.0801 - val_acc: 0.9764
Epoch 7/10
60000/60000 [==============================] - 1s - loss: 0.0571 - acc: 0.9816 - val_loss: 0.0836 - val_acc: 0.9781
Epoch 8/10
60000/60000 [==============================] - 2s 

# Final Results:

The best results (98.15% accuracy) came from using the original MLP activation layers and parameters with a batch_size of 256.